<a href="https://colab.research.google.com/github/DeepthiVReddy/Bike-Sharing/blob/main/Hourly_Bike_Demand_Forecasting_Tree_Based_Regression_and_Model_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the training dataset
train_data = pd.read_csv('Hour_train_validate.csv')

# Load the test dataset for evaluation
test_data = pd.read_csv('Hour_test (1).csv')

In [ ]:
# Check for missing values in the training dataset
print(train_data.isnull().sum())
print(test_data.isnull().sum())

# Drop rows with missing values (if any)
train_data = train_data.dropna()
test_data = test_data.dropna()

# One-hot encoding for categorical variables in both datasets without 'holiday'
train_data = pd.get_dummies(train_data, columns=['weathersit'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['weathersit'], drop_first=True)

# Ensure both datasets have the same columns after one-hot encoding
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)


instant       0
dteday        0
hr            0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64
instant       0
dteday        0
hr            0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64


In [ ]:
# Select features and target variable from the training dataset
X_train = train_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_train = train_data['cnt']

# Select features and target variable from the test dataset
X_test = test_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_test = test_data['cnt']  # We will use 'cnt' for evaluation

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Initialize and train the decision tree model
decision_tree = DecisionTreeRegressor(max_depth=15, min_samples_split=10)
decision_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=15, min_samples_split=10)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the random forest model
random_forest = RandomForestRegressor(n_estimators=200, max_depth=50)
random_forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=50, n_estimators=200)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize and train the gradient boosting model
gradient_boosting = GradientBoostingRegressor(n_estimators=100, max_depth=10)
gradient_boosting.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=10)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [40, 50, 60]
}

# Initialize the grid search with cv set to None for one-fold cross-validation
grid_search_rf = GridSearchCV(RandomForestRegressor(), param_grid, cv=None, scoring='neg_mean_absolute_error')
grid_search_rf.fit(X_train, y_train)

# Print the best parameters
print("Best parameters for Random Forest:", grid_search_rf.best_params_)


Best parameters for Random Forest: {'max_depth': 40, 'n_estimators': 300}


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Use the best model from grid search for prediction
best_rf_model = grid_search_rf.best_estimator_
y_pred = best_rf_model.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f'MAPE on Hour_test (1).csv: {mape:.2f}%')

MAPE on Hour_test (1).csv: 66.62%


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Load the datasets
train_data = pd.read_csv('Hour_train_validate.csv')
test_data = pd.read_csv('Hour_test (1).csv')

# Data cleaning and preprocessing
train_data = train_data.dropna()
test_data = test_data.dropna()

# One-hot encoding for categorical variables in both datasets
train_data = pd.get_dummies(train_data, columns=['weathersit'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['weathersit'], drop_first=True)

# Ensure both datasets have the same columns after one-hot encoding
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Select features and target variable
X_train = train_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_train = train_data['cnt']
X_test = test_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_test = test_data['cnt']

# 1. Decision Tree Model
decision_tree = DecisionTreeRegressor(max_depth=15, min_samples_split=10)
decision_tree.fit(X_train, y_train)
y_pred_dt = decision_tree.predict(X_test)
mape_dt = mean_absolute_percentage_error(y_test, y_pred_dt) * 100

# 2. Random Forest Model
random_forest = RandomForestRegressor(n_estimators=200, max_depth=50)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_test)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf) * 100

# 3. Gradient Boosted Trees Model
gradient_boosting = GradientBoostingRegressor(n_estimators=100, max_depth=10)
gradient_boosting.fit(X_train, y_train)
y_pred_gb = gradient_boosting.predict(X_test)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb) * 100

# Output MAPE values for each model
print(f'MAPE for Decision Tree: {mape_dt:.2f}%')
print(f'MAPE for Random Forest: {mape_rf:.2f}%')
print(f'MAPE for Gradient Boosted Trees: {mape_gb:.2f}%')


MAPE for Decision Tree: 70.71%
MAPE for Random Forest: 66.26%
MAPE for Gradient Boosted Trees: 67.56%


In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_param_dist = {
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10, 15]
}
random_search_dt = RandomizedSearchCV(
    DecisionTreeRegressor(random_state=42),
    dt_param_dist,
    n_iter=10,
    cv=3,
    scoring='neg_mean_absolute_error',
    random_state=42
)
random_search_dt.fit(X_train, y_train)
best_dt_model = random_search_dt.best_estimator_
y_pred_dt = best_dt_model.predict(X_test)
mape_dt = mean_absolute_percentage_error(y_test, y_pred_dt) * 100
print(f'MAPE for optimized Decision Tree: {mape_dt:.2f}%')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


MAPE for optimized Decision Tree: 65.03%


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10]
}
random_search_gb = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42),
    gb_param_dist,
    n_iter=10,
    cv=3,
    scoring='neg_mean_absolute_error',
    random_state=42
)
random_search_gb.fit(X_train, y_train)
best_gb_model = random_search_gb.best_estimator_
y_pred_gb = best_gb_model.predict(X_test)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb) * 100
print(f'MAPE for optimized Gradient Boosted Trees: {mape_gb:.2f}%')

MAPE for optimized Gradient Boosted Trees: 62.88%


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Step 1: Load the Datasets
train_data = pd.read_csv('Hour_train_validate.csv')
test_data = pd.read_csv('Hour_test (1).csv')

# Step 2: Data Cleaning and Preprocessing
train_data = train_data.dropna()
test_data = test_data.dropna()

# One-hot encoding for categorical variables (only weathersit)
train_data = pd.get_dummies(train_data, columns=['weathersit'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['weathersit'], drop_first=True)

# Align the training and test datasets after encoding
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

# Step 3: Feature Selection
X_train = train_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_train = train_data['cnt']

X_test = test_data.drop(['cnt', 'casual', 'registered', 'dteday', 'instant'], axis=1)
y_test = test_data['cnt']

# Step 4: Model Training and Hyperparameter Tuning

# 1. Decision Tree
dt_param_dist = {
    'max_depth': [10, 15, 20],
    'min_samples_split': [5, 10, 15, 20]
}
dt_random_search = RandomizedSearchCV(
    DecisionTreeRegressor(random_state=42),
    dt_param_dist,
    n_iter=10,  # Reduce the number of iterations for quicker tuning
    cv=None,    # One-fold cross-validation
    scoring='neg_mean_absolute_error',
    random_state=42
)
dt_random_search.fit(X_train, y_train)

# Best Decision Tree Model
best_dt_model = dt_random_search.best_estimator_
y_pred_dt = best_dt_model.predict(X_test)
mape_dt = mean_absolute_percentage_error(y_test, y_pred_dt) * 100
print(f'MAPE for optimized Decision Tree: {mape_dt:.2f}%')

# 2. Random Forest
rf_param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [40, 50, 60]
}
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    rf_param_dist,
    n_iter=10,  # Reduce the number of iterations for quicker tuning
    cv=None,    # One-fold cross-validation
    scoring='neg_mean_absolute_error',
    random_state=42
)
rf_random_search.fit(X_train, y_train)

# Best Random Forest Model
best_rf_model = rf_random_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf) * 100
print(f'MAPE for optimized Random Forest: {mape_rf:.2f}%')

# 3. Gradient Boosted Trees
gb_param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15]
}
gb_random_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42),
    gb_param_dist,
    n_iter=10,  # Reduce the number of iterations for quicker tuning
    cv=None,    # One-fold cross-validation
    scoring='neg_mean_absolute_error',
    random_state=42
)
gb_random_search.fit(X_train, y_train)

# Best Gradient Boosted Trees Model
best_gb_model = gb_random_search.best_estimator_
y_pred_gb = best_gb_model.predict(X_test)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb) * 100
print(f'MAPE for optimized Gradient Boosted Trees: {mape_gb:.2f}%')


MAPE for optimized Decision Tree: 65.03%
MAPE for optimized Random Forest: 66.68%
MAPE for optimized Gradient Boosted Trees: 65.73%
